In [ ]:
!pip install PyMySQL

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

**Acessando a instância**

In [ ]:
db_host = '35.239.52.63'
db_user = 'Master'
db_password = 'chico123'
db_name = 'Asset_Maneger_Relacional'

In [ ]:
conn = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

**Extraindo os dados e Carregando dados ações**

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL da página da Wikipedia com a lista das ações NASDAQ-100
url = "https://en.wikipedia.org/wiki/NASDAQ-100"

# Fazer o download da página
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Encontrar a tabela das ações NASDAQ-100
table = soup.find("table", {"id": "constituents"})

# Extrair os tickers da tabela
tickers = []
for row in table.find_all("tr")[1:]:
    ticker = row.find_all("td")[1].text.strip()
    tickers.append(ticker)

# Lista para armazenar dados das ações
stocks_data = []

# Coletar dados das ações
for ticker in tickers:
    stock = yf.Ticker(ticker)
    info = stock.info
    stocks_data.append({
        "Ticker": ticker,
        "Nome": info.get("shortName"),
        "Setor": info.get("sector"),
        "Indústria": info.get("industry"),
        "Capitalização_de_Mercado": info.get("marketCap"),
        "Preço_Atual": info.get("currentPrice"),
        "Beta": info.get("beta"),
        "P_L": info.get("forwardPE")
    })


df_ações = pd.DataFrame(stocks_data)




In [ ]:
df_ações

,Ticker,Nome,Setor,Indústria,Capitalização_de_Mercado,Preço_Atual,Beta,P_L
0,ADBE,Adobe Inc.,Technology,Software - Infrastructure,240517890048,542.44,1.297,28.655045
1,ABNB,"Airbnb, Inc.",Consumer Cyclical,Travel Services,88941649920,140.10,1.190,29.494738
2,GOOGL,Alphabet Inc.,Communication Services,Internet Content & Information,2065105289216,167.00,1.046,19.173365
3,GOOG,Alphabet Inc.,Communication Services,Internet Content & Information,2065216569344,168.68,1.046,21.032417
4,AMZN,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,1899204444160,182.50,1.155,34.112150
...,...,...,...,...,...,...,...,...
96,VRTX,Vertex Pharmaceuticals Incorpor,Healthcare,Biotechnology,127803326464,495.26,0.392,27.423037
97,WBD,"Warner Bros. Discovery, Inc. -",Communication Services,Entertainment,20362076160,8.31,1.497,-118.714294
98,WDAY,"Workday, Inc.",Technology,Software - Application,60679684096,228.64,1.354,29.126116
99,XEL,Xcel Energy Inc.,Utilities,Utilities - Regulated Electric,31871451136,57.36,0.378,14.937501


In [ ]:
df_ações.to_sql(con=conn, name='ações', if_exists='append' ,index=False)

101

Extraindo e carregando dados renda fixa

In [ ]:
import yfinance as yf
import pandas as pd

# Lista de tickers dos principais ETFs de títulos de renda fixa dos EUA
fixed_income_etfs = [
    "TLT",  # iShares 20+ Year Treasury Bond ETF
    "IEF",  # iShares 7-10 Year Treasury Bond ETF
    "SHY",  # iShares 1-3 Year Treasury Bond ETF
    "LQD",  # iShares iBoxx $ Investment Grade Corporate Bond ETF
    "BND",  # Vanguard Total Bond Market ETF
    "AGG",  # iShares Core U.S. Aggregate Bond ETF
    "HYG",  # iShares iBoxx $ High Yield Corporate Bond ETF
    "MUB"   # iShares National Muni Bond ETF
]

# Lista para armazenar dados dos ETFs de renda fixa
fixed_income_data = []

# Coletar dados dos ETFs de renda fixa
for ticker in fixed_income_etfs:
    bond_etf = yf.Ticker(ticker)
    info = bond_etf.info
    history = bond_etf.history(period="1d")
    current_price = history['Close'].iloc[-1] if not history.empty else None
    fixed_income_data.append({
        "Ticker": ticker,
        "Nome": info.get("shortName"),
        "Preço_Atual": current_price
    })

# Converter dados para um DataFrame do pandas
df_renda_fixa = pd.DataFrame(fixed_income_data)




In [ ]:
df_renda_fixa

,Ticker,Nome,Preço_Atual
0,TLT,iShares 20+ Year Treasury Bond,92.989998
1,IEF,iShares 7-10 Year Treasury Bond,95.010002
2,SHY,iShares 1-3 Year Treasury Bond,82.129997
3,LQD,iShares iBoxx $ Investment Grad,108.610001
4,BND,Vanguard Total Bond Market ETF,72.940002
5,AGG,iShares Core U.S. Aggregate Bon,98.320000
6,HYG,iShares iBoxx $ High Yield Corp,78.290001
7,MUB,iShares National Muni Bond ETF,107.290001


In [ ]:
df_renda_fixa.to_sql(con=conn, name='títulos_de_renda_fixa', if_exists='append' ,index=False)

8

Extraindo e carregando dados de commodities

In [ ]:
import yfinance as yf
import pandas as pd

# Lista de tickers das principais commodities
commodities_tickers = [
    "GC=F",  # Gold
    "SI=F",  # Silver
    "CL=F",  # Crude Oil
    "NG=F",  # Natural Gas
    "HG=F",  # Copper
    "ZC=F",  # Corn
    "ZS=F",  # Soybeans
    "ZW=F",  # Wheat
    "LE=F",  # Live Cattle
    "HE=F"   # Lean Hogs
]

# Lista para armazenar dados das commodities
commodities_data = []

# Coletar dados das commodities
for ticker in commodities_tickers:
    commodity = yf.Ticker(ticker)
    info = commodity.info
    history = commodity.history(period="1d")
    current_price = history['Close'].iloc[-1] if not history.empty else None
    commodities_data.append({
        "Ticker": ticker,
        "Nome": info.get("shortName"),
        "Preço_Atual": current_price,
        "Volume": info.get("regularMarketVolume")
    })

# Converter dados para um DataFrame do pandas
df_commodities = pd.DataFrame(commodities_data)



ERROR:yfinance:$LE=F: possibly delisted; No price data found  (period=1d)


$LE=F: possibly delisted; No price data found  (period=1d)


ERROR:yfinance:$HE=F: possibly delisted; No price data found  (period=1d)


$HE=F: possibly delisted; No price data found  (period=1d)


In [ ]:
df_commodities

,Ticker,Nome,Preço_Atual,Volume
0,GC=F,Gold Aug 24,2388.300049,25251
1,SI=F,Silver Sep 24,28.145000,13987
2,CL=F,Crude Oil Sep 24,77.320000,23367
3,NG=F,Natural Gas Aug 24,2.014000,246
4,HG=F,Copper Sep 24,4.113500,7724
5,ZC=F,"Corn Futures,Dec-2024",407.250000,19403
6,ZS=F,"Soybean Futures,Nov-2024",1035.750000,23167
7,ZW=F,"Chicago SRW Wheat Futures,Sep-2",518.500000,5055
8,LE=F,"Live Cattle Futures,Aug-2024",NaN,11014
9,HE=F,"Lean Hogs Futures,Aug-2024",NaN,8002


In [ ]:
df_commodities.to_sql(con=conn, name='commodities', if_exists='append' ,index=False)

10

Extraindo e carregando dados moedas fiduciárias

In [ ]:
import yfinance as yf
import pandas as pd

# Lista de tickers dos principais pares de moedas
currency_pairs = [
    "EURUSD=X",  # Euro to US Dollar
    "GBPUSD=X",  # British Pound to US Dollar
    "USDJPY=X",  # US Dollar to Japanese Yen
    "AUDUSD=X",  # Australian Dollar to US Dollar
    "USDCAD=X",  # US Dollar to Canadian Dollar
    "USDCHF=X",  # US Dollar to Swiss Franc
    "NZDUSD=X",  # New Zealand Dollar to US Dollar
    "USDCNY=X",  # US Dollar to Chinese Yuan
    "USDHKD=X",  # US Dollar to Hong Kong Dollar
    "USDSGD=X"   # US Dollar to Singapore Dollar
]

# Lista para armazenar dados das moedas
currency_data = []

# Coletar dados das moedas
for ticker in currency_pairs:
    currency = yf.Ticker(ticker)
    info = currency.info
    history = currency.history(period="1d")
    current_price = history['Close'].iloc[-1] if not history.empty else None
    currency_data.append({
        "Ticker": ticker,
        "Nome": info.get("shortName"),
        "Preço_Atual": current_price
    })

# Converter dados para um DataFrame do pandas
df_moedas_fiduciárias = pd.DataFrame(currency_data)




In [ ]:
df_moedas_fiduciárias

,Ticker,Nome,Preço_Atual
0,EURUSD=X,EUR/USD,1.085305
1,GBPUSD=X,GBP/USD,1.284720
2,USDJPY=X,USD/JPY,153.423996
3,AUDUSD=X,AUD/USD,0.654819
4,USDCAD=X,USD/CAD,1.383420
5,USDCHF=X,USD/CHF,0.883970
6,NZDUSD=X,NZD/USD,0.588547
7,USDCNY=X,USD/CNY,7.253900
8,USDHKD=X,USD/HKD,7.808670
9,USDSGD=X,USD/SGD,1.342280


In [ ]:
df_moedas_fiduciárias.to_sql(con=conn, name='Moedas_fiduciárias', if_exists='append' ,index=False)

10

Extraindo e carregando dados cripto

In [ ]:
import yfinance as yf
import pandas as pd

# Lista de tickers das principais criptomoedas
crypto_pairs = [
    "BTC-USD",  # Bitcoin to US Dollar
    "ETH-USD",  # Ethereum to US Dollar
    "BNB-USD",  # Binance Coin to US Dollar
    "XRP-USD",  # Ripple to US Dollar
    "ADA-USD",  # Cardano to US Dollar
    "SOL-USD",  # Solana to US Dollar
    "DOGE-USD", # Dogecoin to US Dollar
    "DOT-USD",  # Polkadot to US Dollar
    "LTC-USD",  # Litecoin to US Dollar
    "MATIC-USD" # Polygon to US Dollar
]

# Lista para armazenar dados das criptomoedas
crypto_data = []

# Coletar dados das criptomoedas
for ticker in crypto_pairs:
    crypto = yf.Ticker(ticker)
    info = crypto.info
    market_cap = info.get("marketCap")
    crypto_data.append({
        "Ticker": ticker,
        "Nome": info.get("shortName"),
        "Market_Cap": market_cap
    })

# Converter dados para um DataFrame do pandas
df_cripto = pd.DataFrame(crypto_data)
print(df_cripto)



      Ticker          Nome     Market_Cap
0    BTC-USD   Bitcoin USD  1372320563200
1    ETH-USD  Ethereum USD   405202501632
2    BNB-USD       BNB USD    86604759040
3    XRP-USD       XRP USD    34133886976
4    ADA-USD   Cardano USD    15155334144
5    SOL-USD    Solana USD    89291718656
6   DOGE-USD  Dogecoin USD    19491143680
7    DOT-USD  Polkadot USD     8413948416
8    LTC-USD  Litecoin USD     5506461184
9  MATIC-USD   Polygon USD     5143395840


In [ ]:
df_cripto

,Ticker,Nome,Market_Cap
0,BTC-USD,Bitcoin USD,1372320563200
1,ETH-USD,Ethereum USD,405202501632
2,BNB-USD,BNB USD,86604759040
3,XRP-USD,XRP USD,34133886976
4,ADA-USD,Cardano USD,15155334144
5,SOL-USD,Solana USD,89291718656
6,DOGE-USD,Dogecoin USD,19491143680
7,DOT-USD,Polkadot USD,8413948416
8,LTC-USD,Litecoin USD,5506461184
9,MATIC-USD,Polygon USD,5143395840


In [ ]:
df_cripto.to_sql(con=conn, name='Cripto', if_exists='append' ,index=False)

10

Inserindo os usuários no banco de dados

In [ ]:
df_usuários = pd.read_csv("usuario_o.csv")

In [ ]:
df_usuários.to_sql(con=conn, name='usuário', if_exists='append' ,index=False)

1000

Inserindo o assets commodities

In [ ]:
df_assets_commoddities = pd.read_csv("assets_commodities.csv")

In [ ]:
df_assets_commoddities

,nome,user_id,quantidade
0,"Live Cattle Futures,Aug-2024",74,92
1,Crude Oil Sep 24,474,30
2,Copper Sep 24,863,49
3,"Chicago SRW Wheat Futures,Sep-2",517,44
4,Crude Oil Sep 24,793,41
...,...,...,...
995,Silver Sep 24,898,32
996,Natural Gas Aug 24,516,30
997,Crude Oil Sep 24,132,37
998,"Corn Futures,Dec-2024",244,89


In [ ]:
df_assets_commoddities.to_sql(con=conn, name='Assets_commodities', if_exists='append' ,index=False)

1000

Inserindo o assets fiduciárias

In [ ]:
df_assets_fiduciarias = pd.read_csv("assets_fiduciárias.csv")

In [ ]:
df_assets_fiduciarias.to_sql(con=conn, name='Assets_fiduciárias', if_exists='append' ,index=False)

1000

Inserido o assets_títulos

In [ ]:
df_assets_títulos = pd.read_csv("assets_títulos.csv")


In [ ]:
df_assets_títulos.to_sql(con=conn, name='Assets_titulos', if_exists='append' ,index=False)

1000

Inserindo o assets_ações

In [ ]:
df_assets_ações = pd.read_csv("assets_ações.csv")

In [ ]:
df_assets_ações.to_sql(con=conn, name='Assets_ações', if_exists='append' ,index=False)

1000

Inserindo assets_cripto

In [ ]:
df_assets_cripto = pd.read_csv("assets_critpo.csv")

In [ ]:
df_assets_cripto

,user_id,ticker,quantidade
0,199,SOL-USD,1.245799
1,290,BTC-USD,4.127877
2,665,BTC-USD,1.595334
3,35,ETH-USD,4.763894
4,856,DOGE-USD,2.382318
...,...,...,...
995,110,ADA-USD,1.350789
996,551,MATIC-USD,2.981528
997,815,BNB-USD,4.627911
998,403,XRP-USD,4.517309


In [ ]:
df_assets_cripto.to_sql(con=conn, name='Assets_cripto', if_exists='append' ,index=False)

1000